## part 1 ##

In [1]:
example = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.'''.split('\n')

In [2]:
import re

In [3]:
lhsre = re.compile(r'(\w+ \w+) bag[s]')

In [4]:
m = lhsre.match('dark orange bags')
m.group(1)

'dark orange'

In [5]:
rhsre = re.compile(r'(\d+) (\w+ \w+) bag')

In [6]:
for bag in '2 shiny gold bags, 9 faded blue bags.'.split(','):
    m = rhsre.match(bag.strip())
    print(m.group(1), m.group(2))

2 shiny gold
9 faded blue


In [7]:
example[0].split('contain')

['light red bags ', ' 1 bright white bag, 2 muted yellow bags.']

In [8]:
def parse_bags(lines):
    bags = {}
    for line in lines:
        lhs, rhs = line.split('contain')
        m = lhsre.match(lhs)
        key = m.group(1)
        if 'no other bags' in rhs:
            continue
        d = {}
        for bag in rhs.split(','):
            m = rhsre.match(bag.strip())
            d[m.group(2)] = m.group(1)
        bags[key] = d
    return bags
        

In [9]:
bagmap = parse_bags(example)
bagmap

{'light red': {'bright white': '1', 'muted yellow': '2'},
 'dark orange': {'bright white': '3', 'muted yellow': '4'},
 'bright white': {'shiny gold': '1'},
 'muted yellow': {'shiny gold': '2', 'faded blue': '9'},
 'shiny gold': {'dark olive': '1', 'vibrant plum': '2'},
 'dark olive': {'faded blue': '3', 'dotted black': '4'},
 'vibrant plum': {'faded blue': '5', 'dotted black': '6'}}

In [10]:
def is_item_in_bag(bag, item, bagmap):
    if item in bag:
        return True
    for b in bag:
        if b not in bagmap:
            continue
        newbag = bagmap[b]
        if is_item_in_bag(newbag, item, bagmap):
            return True
    return False

In [11]:
is_item_in_bag(bagmap['bright white'], 'shiny gold', bagmap)

True

In [12]:
is_item_in_bag(bagmap['vibrant plum'], 'shiny gold', bagmap)

False

In [13]:
is_item_in_bag(bagmap['light red'], 'shiny gold', bagmap)

True

In [14]:
sum(is_item_in_bag(bagmap[key], 'shiny gold', bagmap) for key in bagmap)

4

In [15]:
with open('inputs/day7.input') as fp:
    data = fp.readlines()
data[-1]

'pale tan bags contain 5 posh gray bags, 3 wavy violet bags.\n'

In [16]:
data_bagmap = parse_bags(data)

In [17]:
sum(is_item_in_bag(data_bagmap[key], 'shiny gold', data_bagmap) for key in data_bagmap)

326

## part 2 ##

In [18]:
# Note that this function includes the number of bags at the top level, too. To just get contained bags,
# start w/ 1 bag a the top level, and subtract 1 from the total
def count_bags(bag, bagmap):
    sum = 0
    for key in bag:
        num = int(bag[key])
        sum += num
        if key in bagmap:
            sum += num*count_bags(bagmap[key], bagmap)
    return sum 

In [19]:
count_bags({'faded blue': 1}, bagmap) - 1

0

In [20]:
count_bags({'vibrant plum': 1}, bagmap) - 1

11

In [21]:
count_bags({'shiny gold': 1}, bagmap) - 1

32

In [22]:
ex2 = '''shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.'''.split('\n')
bagmap2 = parse_bags(ex2)
bagmap2

{'shiny gold': {'dark red': '2'},
 'dark red': {'dark orange': '2'},
 'dark orange': {'dark yellow': '2'},
 'dark yellow': {'dark green': '2'},
 'dark green': {'dark blue': '2'},
 'dark blue': {'dark violet': '2'}}

In [23]:
count_bags({'shiny gold': 1}, bagmap2) - 1

126

In [24]:
count_bags({'shiny gold': 1}, data_bagmap) - 1

5635